In [1]:
#Loading packages
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
import chess

from keras.models import load_model


In [2]:
#Importing created modules
import os
import sys 

cwd = os.getcwd()
parent_directory = os.path.abspath(os.path.join(cwd, '..', '..'))
sys.path.append(parent_directory)

from utils.move_encoding import encode_move, decode_move
from utils.board_encoding import encode_board, fen_to_board


In [5]:
data_file = "cleaned_data_10K.pkl"
data_path = os.path.join('..', '..', 'data/cleaned_data', data_file)
df_full = pd.read_pickle(data_path)
df = df_full[['board', 'encoded_board', 'move', 'encoded_move']]

,board,encoded_board,move,encoded_move
0,r n b q k b n r\np p p p p p p p\n. . . . . . ...,"[[[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], ...",d7d5,877
1,r n b q k b n r\np p p . p p p p\n. . . . . . ...,"[[[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], ...",c2c4,731
2,r n b q k b n r\np p p . p p p p\n. . . . . . ...,"[[[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], ...",e7e6,803
3,r n b q k b n r\np p p . . p p p\n. . . . p . ...,"[[[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], ...",c4d5,1905
4,r n b q k b n r\np p p . . p p p\n. . . . p . ...,"[[[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], ...",e6d5,1394


In [13]:
df_shape = df['encoded_board'][0].shape

#Additional data cleaning
df = df[df['encoded_move']!=""]


x_max = np.max(df['encoded_board'].apply(lambda x: np.max(x)))
x_min = np.min(df['encoded_board'].apply(lambda x: np.min(x)))
y_max = np.max(df['encoded_move'])

print("Encoded board has a minimum of " + str(x_min) + " and a maximum of " + str(x_max))
print("There are " + str(y_max) + " different encoded moves")

df['encoded_move']


Encoded board has a minimum of 0 and a maximum of 1
There are 4660 different encoded moves


0        877
1        731
2        803
3       1905
4       1394
        ... 
9527    2701
9528    3942
9529    3285
9530    4571
9531    1547
Name: encoded_move, Length: 9532, dtype: object

In [7]:
x = df['encoded_board']
y = df['encoded_move']

x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.3, random_state=42)

x_train = x_train.to_numpy()
x_val = x_val.to_numpy()
y_train = y_train.to_numpy()
y_val = y_val.to_numpy()


In [10]:
# x_train.reshape(6672,8,8,14)

for i in range(len(x_train)):
    x_train[i] = x_train[0].reshape(896)
    
x_train = np.array(x_train.tolist()).astype('float32')

for i in range(len(x_val)):
    x_val[i] = x_val[0].reshape(896)
    
x_val = np.array(x_val.tolist()).astype('float32')

y_train = y_train.astype('float32')
y_val = y_val.astype('float32')

In [7]:
input_shape = x_train[0].shape[0]
y_max = 4672 #Number of possible moves


model = keras.Sequential([
    keras.layers.Flatten(input_shape=(896,)),
    keras.layers.Dense(units=2000, activation='relu'),
    keras.layers.Dense(units=1500, activation='relu'),
    keras.layers.Dense(units=1000, activation='relu'),
    keras.layers.Dense(units=500, activation='relu'),
    keras.layers.Dense(units=500, activation='relu'),
    keras.layers.Dense(units=y_max, activation='softmax')
])

model.compile(
    optimizer='Adam',
    loss='SparseCategoricalCrossentropy', #Because y is an integer representation
    metrics=['accuracy']
)

model.fit(
    x=x_train,
    y=y_train,
    epochs=100
)

Epoch 1/4
2077/2077 [==============================] - 12s 5ms/step - loss: 6.8761 - accuracy: 0.0073
Epoch 2/4
2077/2077 [==============================] - 11s 5ms/step - loss: 6.7484 - accuracy: 0.0083
Epoch 3/4
2077/2077 [==============================] - 11s 5ms/step - loss: 6.7439 - accuracy: 0.0080
Epoch 4/4
2077/2077 [==============================] - 11s 5ms/step - loss: 6.7421 - accuracy: 0.0080


In [32]:
model.save('../saved_models/initial_model.h5')

In [30]:
#Reshape to be able to input to model
prediction_input = x_val[0].reshape(1, 896)
prediction = model(prediction_input)
prediction = prediction.numpy()
prediction = prediction.reshape(4672,)




0.0016174312